In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools
from collections import Counter
import nltk
from nltk.data import load
#tagdict = load('help/tagsets/upenn_tagset.pickle')

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

#import researchpy as rp
import scipy.stats as stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
%%time

# Load 58 Word2Vec models of MPs in T1 & T2

dictOfModels = {}
#folderPath = '/kaggle/input/58alignedmptimemodels/kaggle/working/58-aligned-models-by-mp-and-time'
#folderPath = '/kaggle/input/58mptimemodels/kaggle/working/models-by-mp-and-time'
folderPath = '/kaggle/input/24mptimemodels/kaggle/working/models-by-mp-and-time'

for file in os.listdir(folderPath):
    filePath = folderPath + '/' + file
    #print(filePath)
    if(len(filePath.split('.'))>1):
        continue
    else:
        model = gensim.models.Word2Vec.load(filePath)
        dictOfModels[file] = model

In [ ]:
dictOfModels.keys()

In [ ]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """
    #print('Vocab function')
    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)
    #print('vocabs',list(common_vocab)[0], list(vocab_m1)[0], list(vocab_m2)[0])
    #print('1. Common vocab length', len(common_vocab))

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))
    print('2. Common vocab length', len(common_vocab))
    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
    print('Vocab function returning models with shapes', m1.wv.vectors.shape, m2.wv.vectors.shape)    
    return (m1,m2)

In [ ]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    print('shapes', base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """
    print('Hillow hillow.. In Smart procrustes align gensim function, received',base_embed, other_embed)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    #print('a, the shapes received are', in_base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    #print(str(len(list(in_base_embed.wv.index_to_key))))
    #print(str(len(list(in_other_embed.wv.index_to_key))))

    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
    
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    
    #print('b')

    other_vecs = in_other_embed.wv.get_normed_vectors()
    #print('c')
    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    #print('d')

    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    #print('e')

    # another matrix operation
    ortho = u.dot(v) 
    #print('f')

    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    #print('Procrustes function returning')
    return other_embed

In [ ]:
%%time
modelsToAlign = list(dictOfModels.values())
for i in range(0,len(modelsToAlign)-1):
    functools.reduce(smart_procrustes_align_gensim, modelsToAlign)

In [ ]:
'''# To delete if need be
import shutil
shutil.rmtree('./58-aligned-models-by-mp-and-time')

#os.remove('./file.zip')'''

os.makedirs('./24-aligned-models-by-mp-and-time')

In [ ]:
dictOfModels.keys()
print(dictOfModels['model_df_t1_1516_Labour'])
#print(dictOfModels['model_df_t2_4131'])
#print(dictOfModels['model_df_t2_1423'])

In [ ]:
%%time
# Next step is to save aligned models 

aligned_models_folder = './24-aligned-models-by-mp-and-time'

for c, mod in enumerate(dictOfModels.keys()):
    if(c%10==0):
        print('Saving model number', c+1)
    #modelName = 'aligned_model_' + str(c+1)
    modelName = 'aligned_' + str(mod)

    print(modelName)
    dictOfModels[mod].save(os.path.join(aligned_models_folder, modelName))

In [ ]:
%%time
#  ------- Final list of models saved after alignment ---------

# ------------ new cell --------
!zip -r file.zip /kaggle/working/24-aligned-models-by-mp-and-time
